In [1]:
### import necessary libraries
import ee
import geemap

ee.Authenticate()
ee.Initialize(project="climatech-426614")

In [5]:
# Load the collection of administrative features
admin2 = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level0")

# Filter to Tanzania
tanzania_admin1 = admin2.filter(ee.Filter.eq('ADM0_NAME', 'United Republic of Tanzania'))

# Obtain geometry
tanzania_admin1_geometry = tanzania_admin1.geometry()

# Create an interactive map
Map = geemap.Map()

# Center the map in Tanzania
Map.centerObject(tanzania_admin1_geometry, zoom=6)

Map.addLayer(tanzania_admin1_geometry, {'color': 'blue'}, 'Tanzania')

# Show the map
Map


Map(center=[-6.255852446970605, 34.8049821283269], controls=(WidgetControl(options=['position', 'transparent_b…

In [3]:
# Load the collection of administrative features (level 1)
admin2 = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level1")

# Filter to obtain the first level administrative divisions (provinces/states) of Tanzania.
tanzania_admin1 = admin2.filter(ee.Filter.eq('ADM0_NAME', 'United Republic of Tanzania')).filter(ee.Filter.neq('ADM1_NAME', None))

# Obtain the geometry of the first level administrative divisions of Tanzania.
tanzania_admin1_geometry = tanzania_admin1.geometry()

# Create an interactive map
Map = geemap.Map()

# Center the map in Tanzania
Map.centerObject(tanzania_admin1_geometry, zoom=6)

# Add the layer of first level administrative divisions of Tanzania to the map.
Map.addLayer(tanzania_admin1_geometry, {'color': 'blue'}, 'Tanzania Admin Level 1')

# Show the map
Map

Map(center=[-6.255849750454233, 34.80497931134518], controls=(WidgetControl(options=['position', 'transparent_…

In [6]:
# Load the dataset for level 1 administrative regions
dataset = ee.FeatureCollection('FAO/GAUL_SIMPLIFIED_500m/2015/level1')

# Filter by the country, in this case Tanzania
tanzania = dataset.filter(ee.Filter.eq('ADM0_NAME', 'United Republic of Tanzania'))

# Print the number of features for Tanzania
print('Number of States for Tanzania:', tanzania.size().getInfo())
print(' ')

# Set up the map to visualize the regions
Map = geemap.Map()
Map.centerObject(tanzania, 6)
Map.addLayer(tanzania, {'color': 'blue'}, 'Tanzania Level 1 Administrative Units')

# Display the map (this will only work in environments that support visualization, like Jupyter Notebooks)
# Map.show()  # Uncomment this line if you're in Jupyter Notebooks or similar environments

# Reduce the features to obtain a list of names of the first-level divisions
names_list = tanzania.reduceColumns(ee.Reducer.toList(), ['ADM1_NAME']).get('list')

# Retrieve and print the list of names using getInfo (note that this blocks until the response is received)
names = names_list.getInfo()
names = list(set(names))  # Remove duplicates
names.sort()

# Print names three per line
print('List of States (ADM1) names:')
for i in range(0, len(names), 3):
    print(', '.join(names[i:i+3]))

Number of States for Tanzania: 30
 
List of States (ADM1) names:
Arusha, Dar-es-salaam, Dodoma
Geita, Iringa, Kagera
Kaskazini Pemba, Kaskazini Unguja, Katavi
Kigoma, Kilimanjaro, Kusini Pemba
Kusini Unguja, Lindi, Manyara
Mara, Mbeya, Mjini Magharibi
Morogoro, Mtwara, Mwanza
Njombe, Pwani, Rukwa
Ruvuma, Shinyanga, Simiyu
Singida, Tabora, Tanga


In [4]:
# Load the collection of administrative features (level 2)
admin2 = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level2")

# Filter to obtain the second level administrative divisions (departments) of Tanzania
tanzania_admin1 = admin2.filter(ee.Filter.eq('ADM0_NAME', 'United Republic of Tanzania')).filter(ee.Filter.neq('ADM1_NAME', None))

# Obtain the geometry of the second level administrative divisions of Tanzania.
tanzania_admin1_geometry = tanzania_admin1.geometry()

# Create an interactive map
Map = geemap.Map()

# Center the map in Tanzania
Map.centerObject(tanzania_admin1_geometry, zoom=6)

# Add the layer of second level administrative divisions of Tanzania to the map.
Map.addLayer(tanzania_admin1_geometry, {'color': 'blue'}, 'Tanzania Admin Level 1')

# Show the map
Map

Map(center=[-6.255857589074118, 34.804983840951266], controls=(WidgetControl(options=['position', 'transparent…

In [15]:
# Load the dataset for level 2 administrative regions
dataset_level2 = ee.FeatureCollection('FAO/GAUL_SIMPLIFIED_500m/2015/level2')

# Filter by the country, in this case Tanzania
tanzania_level2 = dataset_level2.filter(ee.Filter.eq('ADM0_NAME', 'United Republic of Tanzania'))

# Print the number of departments for Tanzania
print('Number of departments for Tanzania:', tanzania_level2.size().getInfo())

# Set up the map to visualize the regions
Map = geemap.Map()
Map.centerObject(tanzania_level2, 6)
Map.addLayer(tanzania_level2, {'color': 'green'}, 'Tanzania Level 2 Administrative Units')

# Function to create a structure suitable for grouping
def format_feature(feature):
    return ee.Feature(None, {
        'ADM1_NAME': feature.get('ADM1_NAME'),
        'ADM2_NAME': ee.List([feature.get('ADM2_NAME')])
    })

# Map the function over the FeatureCollection
formatted_features = tanzania_level2.map(format_feature)

# Reduce to a dictionary format by state and concatenate department names
grouped_data = formatted_features.reduceColumns(
    ee.Reducer.toList().group(
        groupField=1,
        groupName='ADM1_NAME'
    ),
    ['ADM1_NAME', 'ADM2_NAME']
)

# Fetch the data to the Python environment
state_dept_info = grouped_data.getInfo()
# Print each state and its departments
for group in state_dept_info['groups']:
    # Print the state name
    print('State:', group['ADM1_NAME'])  # Corrected from 'name' to 'ADM1_NAME'
    departments = []  # Initialize an empty list for departments
    
    # Check if 'list' key exists in the current group
    if 'list' in group:
        departments.extend(group['list'])  # Extend the departments list with the items in 'list'
    
    departments = list(set(departments))  # Remove duplicates
    departments.sort()  # Sort the department names
    
    # Print the departments
    for dept in departments:
        print('  -', dept)
    
    print(' ')  # Print a newline for better readability

# Display the map (this will only work in environments that support visualization, like Jupyter Notebooks)
# Map.show()  # Uncomment this line if you're in Jupyter Notebooks or similar environments


Number of departments for Tanzania: 182
State: ['Karatu']
  - Arusha
 
State: ['Iringa Rural']
  - Iringa
 
State: ['Kilolo']
  - Iringa
 
State: ['Mbulu']
  - Manyara
 
State: ['Mbarali']
  - Mbeya
 
State: ['Mbeya Rural']
  - Mbeya
 
State: ['Newala']
  - Mtwara
 
State: ['Tandahimba']
  - Mtwara
 
State: ['Kwimba']
  - Mwanza
 
State: ['Misungwi']
  - Mwanza
 
State: ['Kisarawe']
  - Pwani
 
State: ['Mkuranga']
  - Pwani
 
State: ['Kahama']
  - Shinyanga
 
State: ['Kishapu']
  - Shinyanga
 
State: ['Shinyanga Rural']
  - Shinyanga
 
State: ['Morogoro']
  - Morogoro
 
State: ['Mvomero']
  - Morogoro
 
State: ['Handeni']
  - Tanga
 
State: ['Handeni Township Authority']
  - Tanga
 
State: ['Kilindi']
  - Tanga
 
State: ['Mkinga']
  - Tanga
 
State: ['Muheza']
  - Tanga
 
State: ['Chemba']
  - Dodoma
 
State: ['Kondoa']
  - Dodoma
 
State: ['Bukombe']
  - Geita
 
State: ['Geita']
  - Geita
 
State: ['Mbogwe']
  - Geita
 
State: ["Nyang'wale"]
  - Geita
 
State: ['Karagwe']
  - Kagera
 